In [ ]:
import pandas as pd
import cx_Oracle
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import folium
from folium.plugins import HeatMap, FeatureGroupSubGroup
import tkinter as tk
from tkinter import messagebox
from tkcalendar import Calendar
from tkinter import messagebox, ttk

In [ ]:

def data_gathering():    
    # Connection string
    dsn = cx_Oracle.makedsn(host='{HOST IP}', port='1521', service_name='{SERVICE NAME}')
    conn = cx_Oracle.connect(user='{USER NAME}', password='{PASSWORD}', dsn=dsn)


    cursor = conn.cursor()
    query = f"""SELECT line.CREATED_DATETIME, line.INVOICE_ID, branch.CODE, item.DESCRIPTION_1, line.INVOICE_QUANTITY, inv.CUSTOMER_ID, loc.DESCRIPTION_1 as store, loc.X_POSITION, loc.Y_POSITION 
    FROM VP.T_INVOICE_LINE line, VP.T_BRANCH branch, VP.T_ITEM item, VP.T_INVOICE inv, VP.T_CUSTOMER_LOCATION loc
    WHERE line.BRANCH_ID = branch.RECORD_ID 
    AND line.ITEM_ID = item.RECORD_ID 
    AND line.INVOICE_ID = inv.RECORD_ID 
    AND inv.CUSTOMER_ID = loc.CUSTOMER_ID 
    AND TRUNC(line.CREATED_DATETIME) = DATE '{selected_date}'
    AND line.BRANCH_ID IN (SELECT RECORD_ID FROM VP.T_BRANCH WHERE CODE LIKE 'SA{brn}%')
    AND item.DESCRIPTION_1 NOT LIKE 'FOC%'
    AND loc.CUSTOMER_ID != 1 """



    # Execute the query with parameter binding
    cursor.execute(query)

    # Fetch all the rows from the cursor
    rows = cursor.fetchall()

    # Get the column names from the cursor description
    columns = [desc[0] for desc in cursor.description]

    # Create a Pandas DataFrame from the rows and columns
    df_result = pd.DataFrame(rows, columns=columns)

    cursor.close()
    conn.close()

    print(df_result.head())
    df_result.to_excel('Inv-detaild.xlsx', index=False)

    
# Calendar and Region Selection
def on_date_selected():
    global selected_date
    selected_date = cal.selection_get()
    if not branch_var.get():
        messagebox.showwarning("Branch Selection", "Please select a branch.")
    else:
        messagebox.showinfo("Date and Branch", f"You selected:\n{selected_date}\nRegion {brn}")
        window.destroy()
        data_gathering()

def on_branch_selected(event):
    global brn
    brn = branch_var.get()

# Create the main application window
window = tk.Tk()
window.title("Date Picker")

# Create a label for the Branch dropdown
branch_label = tk.Label(window, text="Branch:")

# Create a dropdown for Branch selection
branch_var = tk.StringVar()
branch_dropdown = ttk.Combobox(window, textvariable=branch_var)
branch_dropdown['values'] = tuple(range(1, 9))  # Numbers from 1 to 8
branch_dropdown.bind("<<ComboboxSelected>>", on_branch_selected)

# Create a Calendar widget
cal = Calendar(window, selectmode='day')

# Create a button to get the selected date
button = tk.Button(window, text="Generate Report", command=on_date_selected)

# Pack the widgets with spacing
branch_label.pack()
branch_dropdown.pack(pady=5)  # Add vertical spacing
cal.pack(pady=5)  # Add vertical spacing
button.pack()

# Start the main event loop
window.mainloop()



In [ ]:
print(selected_date)
print(brn)

In [ ]:
df = pd.read_excel('Inv-detaild.xlsx', header = 0)
df.info()
df

In [ ]:
df = df.dropna(axis=0)
df.info()
df

In [ ]:
# Choose the coordinates for the center of the map
center_lat = 32.4279
center_lon = 53.6880

# Create the folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

for product in df['DESCRIPTION_1'].unique():
    filtered = df[df['DESCRIPTION_1'] == product]

    heat_data = []
    layer_group = folium.FeatureGroup(name=product, show=False)

    for index, row in filtered.iterrows():
        lat = row['X_POSITION']
        lon = row['Y_POSITION']
        quantity = row['INVOICE_QUANTITY']
        store = row['STORE']
        code = row['CODE']

        heat_data.append([lat, lon, quantity])

        popup_content = f'<strong>Branch:</strong> {code}<br><strong>Name:</strong> {store}<br><strong>QTY:</strong> {quantity} Case(s)'
        
        icon_html = '<i class="fa-solid fa-circle-info fa-lg"></i>'
        icon = folium.DivIcon(html=icon_html)
        
        marker = folium.Marker(location=[lat, lon], icon=icon, popup=folium.Popup(popup_content, max_width=250))
        marker.add_to(layer_group)

    heat_layer = HeatMap(heat_data, name=product, show=False)
    heat_layer.add_to(layer_group)

    layer_group.add_to(m)

folium.LayerControl().add_to(m)
m.save('heatmap.html')
